# Guild AI Interactive Python Quick Start

This Notebook mirrors the steps in [Guild AI Quick Start](https://guild.ai/docs/start/).

Guild AI is a tool for running, tracking, and comparing machine learning experiments. For more on Guild AI, visit https://guild.ai.

In this example, we show how Guild is used to run an unmodified Python function as a training operation. It uses a Pythonic interface defined in the module `guild.ipy`.

This notebook covers the following topics:

- Installation
- Experiment basics
    - Create a mock training function
    - Generate a run with Guild
    - Examine the run
    - Generate a second run
    - Compare runs
- Hyperparameter search and optimization
    - Grid search
    - Random search
    - Bayesian optimization

## Installation

Before continuing, install Guild AI:

    $ pip install guildai
    
Install either `tensorflow` or `tensorflow-gpu` depending on your system capability:

    $ pip install tensorflow # (or tensorflow-gpu if your system has a GPU)
    
TensorFlow is required for TensorBoard support. Your training code does not need to use TensorFlow. This requirement will be removed in versions of Guild starting with `0.7`.
    
**<span style="color:#600">IMPORTANT</span>** If you are installing Guild AI 0.6.3, you must manually install a missing requirement (this bug is fixed in 0.6.4, which will be released in late May):

    $ pip install click
    
## Contents
    
This Notebook has two parts.

It features the module `guild.ipy`, which is a Python interface that can be used interactively in Notebooks or by your own Python code. 

## Notebooks config

Modify the variables below to change Notebook configuration.

`GULID_HOME` is the location of generated runs. By default it is the location subdirectory "guild-env". Note that we initialize this directory below by permanently deleting any runs it contains before proceeding. If you don't want to delete runs in `GUILD_HOME`, set `DELETE_RUNS_ON_INIT` to `False` below.

In [1]:
GUILD_HOME = "guild-env"

`DELETE_RUNS_ON_INIT` determines whether or not runs are initially deleted from `GUILD_HOME` below. As this Notebook is for demonstration purposes, it's usually a good idea to delete any existing runs before proceeding. To prevent any runs from being deleted, set `DELETE_RUNS_ON_INIT` to `False`.

In [2]:
DELETE_RUNS_ON_INIT = True

## Guild interactive Python API

Guild AI functionality is available through a Notebook compatible interface defined in `guild.ipy`.

Import this module as `guild`:

In [3]:
import guild.ipy as guild

**<span style="color:#600">IMPORTANT</span>** If you get an error **No module named 'click'** install `click` manually by running `pip install click` in your Notebook environment.

## Initialize Guild Home

Guild home is a directory containing the runs that Guild generates. This Notebook uses the directory defined by `GUILD_HOME`. Ensure that the directory exists and use `set_guild_home()` to set the value. 

In [4]:
import os

if not os.path.exists(GUILD_HOME):
    os.mkdir("guild-env")
    
guild.set_guild_home("guild-env")

Clear the director of any runs.

In [5]:
if DELETE_RUNS_ON_INIT:
    deleted = guild.runs().delete(permanent=True)
    print("Deleted %i run(s)" % len(deleted))

Deleted 22 run(s)


## Mock training function

Create a mock training script. This function doesn’t actually train anything, but simulates the training process of accepting hyperparameters as inputs and generating a loss.

In [6]:
import numpy as np

def train(x=0.1, noise=0.1):
    loss = (np.sin(5 * x) * (1 - np.tanh(x ** 2)) + np.random.randn() * noise)
    print("loss: %f" % loss)

[Function credit: *skopt API documentation*](https://scikit-optimize.github.io/)

Run the mock training function a couple times to see how it works.

**NOTE** The function uses a random component to simulate training "noise". This causes the results to differ across runs.

In [7]:
print("Trial 1:")
train(x=-2.0)

Trial 1:
loss: 0.034587


In [8]:
print("Trial 2:")
train(x=0.0)

Trial 2:
loss: 0.071008


In [9]:
print("Trial 3:")
train(x=2.0)

Trial 3:
loss: -0.037958


## Generate a run

Run the mock train function using Guild:

In [10]:
run, return_val = guild.run(train)

loss: 0.324545


The `run()` function returns a tuple of the Guild run and the return value of the function.

In this case, our mock training function doesn't return a value.

In [11]:
print(return_val)

None


The `run` variable is a Python object that represents the run.

## Examine the run

We can work with the run object directly to get information about it.

Each run has a uniqiue ID:

In [12]:
run.id

'5ed5c0327e7911e9b16de4a471939b0d'

Each run is associated with a unique directory, which includes the ID. The run directory contains metadata and output associated with the run.

In [13]:
run.dir

'guild-env/runs/5ed5c0327e7911e9b16de4a471939b0d'

Our mock training script doesn't generate any files, so the run directory is empty, with the exception of a `.guild` subdirectory.

In [14]:
os.listdir(run.dir)

['.guild']

The `.guild` subdirectory contains run output and metadata. Outputs include the output printed to the script during the operation (`output`) and any scalars logged (`events.*`). Guild logs scalar output using TensorFlow event files, which can be read using TensorBoard.

In [15]:
os.listdir(os.path.join(run.dir, ".guild"))

['output', 'opref', 'events.out.tfevents.1558739535.omaha', 'attrs']

Run metadata includes *attributes*. You can list the attribute names using `attr_names()`:

In [16]:
run.attr_names()

['exit_status', 'flags', 'id', 'initialized', 'started', 'stopped']

Read an attribute value using `get()`. For example, to read the *flags* attribute, use:

In [17]:
run.get("flags")

{'noise': 0.1, 'x': 0.1}

Flags are values provided to the function. In this case, both values are defined as function keyword default values. Later you run `train` using explicit flag values.

Each run has a *status*, which indicates if the run is still running and whether or not it completed successfully or terminated with an error.

In [18]:
run.status

'completed'

## List runs

List runs using `guild.runs()`:

In [19]:
runs = guild.runs()
runs

,run,operation,started,status,label
0,5ed5c032,train(),2019-05-24 18:12:15,completed,


Each time you run an operation, the run appears in the data frame generated by `runs()`.

## Print run info

From `runs`, we can print information for the latest run using `runs.info()`.

In [20]:
runs.info()

id: 5ed5c0327e7911e9b16de4a471939b0d
operation: train()
status: completed
started: 2019-05-24 18:12:15
stopped: 2019-05-24 18:12:15
label: 
run_dir: ~/SCM/guild-examples/notebooks/guild-env/runs/5ed5c0327e7911e9b16de4a471939b0d
flags:
  noise: 0.1
  x: 0.1


This information reflects the information you saw in the prior section.

## Run scalars

Run results are recorded as *scalars*. You can list scalars in two ways.

First, you can specify the `scalars` flag to `info()`:

In [21]:
runs.info(scalars=True)

id: 5ed5c0327e7911e9b16de4a471939b0d
operation: train()
status: completed
started: 2019-05-24 18:12:15
stopped: 2019-05-24 18:12:15
label: 
run_dir: ~/SCM/guild-examples/notebooks/guild-env/runs/5ed5c0327e7911e9b16de4a471939b0d
flags:
  noise: 0.1
  x: 0.1
scalars:
  loss: 0.324545 (step 0)


Second, you can get a data frame containing scalars using the `scalars()` method.

In [22]:
scalars = runs.scalars()
scalars

,avg_val,count,first_step,first_val,last_step,last_val,max_step,max_val,min_step,min_val,prefix,run,tag,total
0,0.324545,1,0,0.324545,0,0.324545,0,0.324545,0,0.324545,,5ed5c0327e7911e9b16de4a471939b0d,loss,0.324545


**<span style="color:#600">IMPORTANT</span>** If this listing is empty, ensure that you have TensorFlow installed (see above for instructions).

Guild stores aggregates of each tag, including *first*, *last*, *max*, *min*, and *average*.

You can use the various facilities in data frame to get scalar values.

In [23]:
scalars.query("(run == '%s') and (tag == 'loss')" % run.id)["last_val"]

0    0.324545
Name: last_val, dtype: float64

## Generate a second run

Run train again with different flags along with a run *label*:

In [24]:
_ = guild.run(train, x=0.2, _label="run 2")

loss: 0.855898


List runs:

In [25]:
guild.runs()

,run,operation,started,status,label
0,8fc49fc4,train(),2019-05-24 18:13:37,completed,run 2
1,5ed5c032,train(),2019-05-24 18:12:15,completed,


Note that the latest run contains the label "run 2" as specified in the `run()` call.

## Compare runs

Use `compare()` to generate a data frame containing both flags and scalars.

In [26]:
guild.runs().compare()

,run,operation,started,time,status,label,noise,x,step,loss
0,8fc49fc4,train(),2019-05-24 18:13:37,0:00:00,completed,run 2,0.1,0.2,0,0.855898
1,5ed5c032,train(),2019-05-24 18:12:15,0:00:00,completed,,0.1,0.1,0,0.324545


## Grid search

Grid search &mdash; also referred to as a parameter sweep &mdash; is a form of hyperparameter tuning that uses exhaustive search over a manually defined set of hyperparameter values.

To perform a grid search in Guild, provide a list of values to use for any given flag. If you specify lists for multiple flags, Guild runs trials for each possible flag value combination.

Run `train()` using a range of values:

In [27]:
_ = guild.run(train, x=[-0.5,-0.4,-0.3,-0.2,-0.1])

Running train (noise=0.1, x=-0.5):
loss: -0.370670
Running train (noise=0.1, x=-0.4):
loss: -0.930821
Running train (noise=0.1, x=-0.3):
loss: -0.913095
Running train (noise=0.1, x=-0.2):
loss: -0.856592
Running train (noise=0.1, x=-0.1):
loss: -0.333937


This command generates five trials, one for each specified value of `x`.

Use `runs().compare()[:5]` to compare the last five results:

In [28]:
guild.runs().compare()[:5]

,run,operation,started,time,status,label,noise,x,step,loss
0,91a006a2,train(),2019-05-24 18:13:41,0:00:00,completed,,0.1,-0.1,0,-0.333937
1,91a006a1,train(),2019-05-24 18:13:41,0:00:00,completed,,0.1,-0.2,0,-0.856592
2,91a006a0,train(),2019-05-24 18:13:41,0:00:00,completed,,0.1,-0.3,0,-0.913095
3,91a0069f,train(),2019-05-24 18:13:41,0:00:00,completed,,0.1,-0.4,0,-0.930821
4,91a0069e,train(),2019-05-24 18:13:41,0:00:00,completed,,0.1,-0.5,0,-0.370670


View the top-three results by loss:

In [29]:
guild.runs().compare().sort_values(by="loss")[:3]

,run,operation,started,time,status,label,noise,x,step,loss
3,91a0069f,train(),2019-05-24 18:13:41,0:00:00,completed,,0.1,-0.4,0,-0.930821
2,91a006a0,train(),2019-05-24 18:13:41,0:00:00,completed,,0.1,-0.3,0,-0.913095
1,91a006a1,train(),2019-05-24 18:13:41,0:00:00,completed,,0.1,-0.2,0,-0.856592


Based on our mock training function, the "best" result (i.e. the run with the lowest *loss*) should be the run where `x` is close to `-0.3`. Because there's a random component (i.e. the `noise` parameter) your results may show best results with different values for `x`.

Below is an image that plots *loss* for values of *x*, showing the lowest loss where x is approximately `-0.3`.

<img src="bayesian-optimization.png" style="margin-left:0">

[Image credit: *Bayesian optimization with skopt*](https://scikit-optimize.github.io/notebooks/bayesian-optimization.html)

## Random search

Random search is a method used in machine learning to explore hyperparameter spaces at random.

To run a series of runs using random values over a specified range, use the `slice` function to specify a range for a flag. To specify the number of trials, specify `_max_trials`. The default number of trials is `20`.

Run train five times with random values of `x` over the range `-2.0` to `2.0`.

In [30]:
_ = guild.run(train, x=slice(-2.0,2.0), _max_trials=5)

Running train (noise=0.1, x=0.2561135870353879):
loss: 0.867373
Running train (noise=0.1, x=-1.0868066550723305):
loss: 0.316093
Running train (noise=0.1, x=0.7326336951105894):
loss: -0.300587
Running train (noise=0.1, x=-1.5534667926821455):
loss: -0.030411
Running train (noise=0.1, x=-0.3237612471656277):
loss: -0.891646


Compare the results again.

In [31]:
guild.runs().compare()

,run,operation,started,time,status,label,noise,x,step,loss
0,93df4018,train(),2019-05-24 18:13:44,0:00:00,completed,random,0.1,-0.323761,0,-0.891646
1,93df4017,train(),2019-05-24 18:13:44,0:00:00,completed,random,0.1,-1.553467,0,-0.030411
2,93df4016,train(),2019-05-24 18:13:44,0:00:00,completed,random,0.1,0.732634,0,-0.300587
3,93df4015,train(),2019-05-24 18:13:44,0:00:00,completed,random,0.1,-1.086807,0,0.316093
4,93df4014,train(),2019-05-24 18:13:44,0:00:00,completed,random,0.1,0.256114,0,0.867373
5,91a006a2,train(),2019-05-24 18:13:41,0:00:00,completed,,0.1,-0.100000,0,-0.333937
6,91a006a1,train(),2019-05-24 18:13:41,0:00:00,completed,,0.1,-0.200000,0,-0.856592
7,91a006a0,train(),2019-05-24 18:13:41,0:00:00,completed,,0.1,-0.300000,0,-0.913095
8,91a0069f,train(),2019-05-24 18:13:41,0:00:00,completed,,0.1,-0.400000,0,-0.930821
9,91a0069e,train(),2019-05-24 18:13:41,0:00:00,completed,,0.1,-0.500000,0,-0.370670


List the three runs with the lowest loss:

In [32]:
guild.runs().compare().sort_values(by="loss")[:3]

,run,operation,started,time,status,label,noise,x,step,loss
8,91a0069f,train(),2019-05-24 18:13:41,0:00:00,completed,,0.1,-0.400000,0,-0.930821
7,91a006a0,train(),2019-05-24 18:13:41,0:00:00,completed,,0.1,-0.300000,0,-0.913095
0,93df4018,train(),2019-05-24 18:13:44,0:00:00,completed,random,0.1,-0.323761,0,-0.891646


## Bayesian optimization

Bayesian optimizers use light weight models as surrogates for the target model — surrogates that can be evaluated quickly to recommend likely optimal hyperparameter values — and update those models using results from real trials.

Run `train` with Guild’s built-in Bayesian optimizer, which uses Gaussian processes. As with the earlier random search, we use Python's slice function to specify the range for `x` over which to search.

**NOTE:** The argument `bayesian` in the command is an alias for `gp`, which is a Bayesian optimizer that uses Gaussian processes. Guild supports three Bayesian optimizers: gp, forest and gbrt.

In [33]:
_ = guild.run(train, x=slice(-2.0,2.0), _optimizer="bayesian", _max_trials=10)

Running train (noise=0.1, x=-0.5564354308743733):
loss: -0.163120
Running train (noise=0.1, x=2.0):
loss: 0.093161
Running train (noise=0.1, x=-0.5203875452004019):
loss: -0.276288
Running train (noise=0.1, x=-0.4475056321579718):
loss: -0.539672
Running train (noise=0.1, x=-0.34464229852402806):
loss: -0.912400
Running train (noise=0.1, x=0.29486329659190424):
loss: 0.901384
Running train (noise=0.1, x=-2.0):
loss: -0.057347
Running train (noise=0.1, x=-0.16395954477356667):
loss: -0.639217
Running train (noise=0.1, x=-0.2932673048922452):
loss: -0.952961
Running train (noise=0.1, x=1.2582331207865822):
loss: 0.027738


View the top 10 results:

In [34]:
guild.runs().compare().sort_values(by="loss")[:10]

,run,operation,started,time,status,label,noise,x,step,loss
1,997d9cdd,train(),2019-05-24 18:13:54,0:00:00,completed,gp,0.1,-0.293267,0,-0.952961
18,91a0069f,train(),2019-05-24 18:13:41,0:00:00,completed,,0.1,-0.400000,0,-0.930821
17,91a006a0,train(),2019-05-24 18:13:41,0:00:00,completed,,0.1,-0.300000,0,-0.913095
5,9827ca75,train(),2019-05-24 18:13:52,0:00:00,completed,gp,0.1,-0.344642,0,-0.912400
10,93df4018,train(),2019-05-24 18:13:44,0:00:00,completed,random,0.1,-0.323761,0,-0.891646
16,91a006a1,train(),2019-05-24 18:13:41,0:00:00,completed,,0.1,-0.200000,0,-0.856592
2,997d9cdc,train(),2019-05-24 18:13:54,0:00:00,completed,gp,0.1,-0.163960,0,-0.639217
6,9827ca74,train(),2019-05-24 18:13:52,0:00:00,completed,gp,0.1,-0.447506,0,-0.539672
19,91a0069e,train(),2019-05-24 18:13:41,0:00:00,completed,,0.1,-0.500000,0,-0.370670
15,91a006a2,train(),2019-05-24 18:13:41,0:00:00,completed,,0.1,-0.100000,0,-0.333937


## Next steps

This Notebook covers basic functionality provided by the `guild.ipy` module. The module lets you run functions and capture results as unique runs. You can view and compare run results using various module functions.

Generated runs can be further managed using the Guild command line interface. For more information, refer to [Guild AI Quick Start](https://guild.ai/docs/start/).